<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

[課題] QTL-seq入門 (2): リードアライメント、SNP検出
---

　課題をはじめる前に、次のコードセルを一度実行してください。課題に必要なライブラリのインストールや、プログラム、サンプルファイルをダウンロードします。

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
%%bash
## ソフトウェアのインストール
wget -q -O install.sh https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/scripts/install_for_alignment_hw.sh
sh install.sh && rm -f install.sh
pip install -q igv-notebook==0.3.1
pip list | grep -e "igv-notebook"
## サンプルファイルのダウンロード
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py -O igv_prep.py
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/hw6.tar.gz -O - | tar -zxf - 2>/dev/null

# 課題1

　すぐ下のプログラムを実行すると、品種Hのゲノム配列を参照配列にした品種Lのリードアライメント結果が表示されます。
- 参照配列: 品種H（形質値: 「High」）
- リード配列: 品種L（形質値: 「Low」）

　品種Hと品種Lの間のSNPとみなせる箇所は何箇所あるでしょうか？

In [ ]:
## 課題1
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
ref = RefTrack({ "fastaPath":"hw6/CultivarH.fa", "indexPath":"hw6/CultivarH.fa.fai", "id":"CultivarH" })
B = BamTrack({ "name":"CultivarL", "path":"hw6/CultivarL.bam", "indexPath":"hw6/CultivarL.bam.bai", "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(B)

In [ ]:
#@title { vertical-output: true, display-mode: "form" }
Answer = "" #@param {type:"string"}

### 解答例

13箇所

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/L06_ans_1.png?raw=true" alt="L06_ans_1" height="200px" align="middle">

# 課題2

　（課題1で比較した）品種Hと品種Lの交配し、F<small>2</small>世代の30個体集団を作りました。この集団には、「High」と「Medium」、「Low」の3つの形質値が見られました。

　形質の原因変異を特定するために、全F<small>2</small>個体のうち、形質値「High」をもつ4個体と形質値「Low」をもつ4個体を使って、バルクシーケンシングをおこないました。
- H-バルク: 形質値「High」をもつF<small>2</small>世代4個体のバルク
- L-バルク: 形質値「Low」をもつF<small>2</small>世代4個体のバルク

　形質の違いを生み出している原因変異はどのあたりにあるでしょうか？　次のプログラムを実行して、バルクシーケンシングのアライメント結果を調査し、回答してください。

<small>※ ピンポイントな塩基の位置を答える必要はなく、原因変異がありそうな範囲（先頭から数えて1番〜10番塩基の範囲、など）を回答してもOKです。</small>

In [ ]:
## 課題2
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
import re
def Pooling(Ns, bam_dir, samplingRate=-1, output="bulk.bam", verbose=False, samtools_path="samtools"):
    f2_list = [bam_dir+"/F2_{:03}.bam".format(f) for f in sorted(list(set(Ns)))]
    if samplingRate>=1.0:
        samplingRate = 1.0
    elif samplingRate<=0.0:
        samplingRate = float(1.0 / len(bam_dir) * 5.0)
    f2_str = " ".join(f2_list)
    !$samtools_path merge -o - $f2_str | samtools view -b -s $samplingRate > $output
    !$samtools_path index $output
    if verbose:
      f2_samples = [re.search(r".*/(.+?)$", i).group(1) for i in f2_list]
      print("Output: ", output)
      print("- Reads sampled from: "+",".join(f2_samples))
      print("- Sampling rate: %.3f" % samplingRate)
H_bulk = [8, 9, 14, 22]
L_bulk = [2, 4, 15, 16]
samplingRate = 0.3
Pooling(H_bulk, bam_dir="hw6", output="H_bulk.bam", samplingRate=samplingRate, verbose=True)
Pooling(L_bulk, bam_dir="hw6", output="L_bulk.bam", samplingRate=samplingRate, verbose=True)
ref = RefTrack({ "fastaPath":"hw6/CultivarH.fa", "indexPath":"hw6/CultivarH.fa.fai", "id":"CultivarH" })
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":200 }
H = BamTrack({ **{"name":"H-bulk", "path":"H_bulk.bam", "indexPath":"H_bulk.bam.bai"}, **bam_common_info })
L = BamTrack({ **{"name":"L-bulk", "path":"L_bulk.bam", "indexPath":"L_bulk.bam.bai"}, **bam_common_info })
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(H)
b.load_track(L)

In [ ]:
#@title { vertical-output: true, display-mode: "form" }
Answer = "" #@param {type:"string"}

### 解答例



　（198、）214、233、270番目の塩基を含む範囲

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/L06_ans_2.png?raw=true" alt="L06_ans_2" height="220px" align="middle">

# 課題3

　今回の課題サンプルデータの場合、原因変異をピンポイントで特定するためには、次のどちらが有効でしょうか？

1. より多くの個体をバルクに含める
2. バルクシーケンシングで得るリード量を増やす

　次のプログラムの`H_bulk`、`L_bulk`、`samplingRate`の値を変更してアライメント結果を調べ、回答してください。

- H-バルクに含める個体数を変更する: `H_bulk`の右辺を変更する
  - 個体番号を列挙する際、番号間にはカンマ記号(`,`)が必要
  - 個体番号は半角数字を入力する必要がある
  - 初期値: `H_bulk = [8, 9, 14, 22]`
- L-バルクに含める個体数を変更する: `L_bulk`の右辺を変更する
  - 個体番号を列挙する際、番号間にはカンマ記号(`,`)が必要
  - 個体番号は半角数字を入力する必要がある
  - 初期値: `L_bulk = [2, 4, 15, 16]`
- リード量を増やす: `samplingRate`の右辺を変更する
  - 0.0-1.0までの数値を入力可能
  - 半角数字を入力する必要がある
  - 初期値: `samplingRate = 0.3`


F<small>2</small>個体の形質情報: 
- High: 8, 9, 11, 14, 22, 24, 25, 26
- Medium: 3, 5, 7, 10, 12, 13, 17, 19, 20, 21, 29, 30
- Low: 1, 2, 4, 6, 15, 16, 18, 23, 27, 28
- [形質データリスト](https://github.com/CropEvol/lecture/blob/master/textbook_2022/datasets/f2_pheno_hw.pdf)


In [ ]:
## 課題3
##---- 値を変更可能 -------------
H_bulk = [8, 9, 14, 22, 24]   # H-バルクの個体番号
L_bulk = [2, 4, 15, 16, 18]   # L-バルクの個体番号
samplingRate = 0.3        # バルクリード量（0.0~1.0の範囲）
##------------------------------------
Pooling(H_bulk, bam_dir="hw6", output="H_bulk.bam", samplingRate=samplingRate, verbose=True)
Pooling(L_bulk, bam_dir="hw6", output="L_bulk.bam", samplingRate=samplingRate, verbose=True)
ref = RefTrack({ "fastaPath":"hw6/CultivarH.fa", "indexPath":"hw6/CultivarH.fa.fai", "id":"CultivarH" })
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":200 }
H = BamTrack({ **{"name":"H-bulk", "path":"H_bulk.bam", "indexPath":"H_bulk.bam.bai"}, **bam_common_info })
L = BamTrack({ **{"name":"L-bulk", "path":"L_bulk.bam", "indexPath":"L_bulk.bam.bai"}, **bam_common_info })
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(H)
b.load_track(L)

In [ ]:
#@title { vertical-output: true, display-mode: "form" }
Answer = "" #@param {type:"string"}

### 解答例



　リード数を最大値（`samplingRate = 1.0`）に設定しても、233番目と270番目の塩基どちらかが原因変異か絞り込むことはできない。バルクに含める個体数を増やすと（`H_bulk`や`L_bulk`に個体番号を追加すると）、270番目の塩基が両バルクで対照的な固定SNPをもつことを観察できる。

　したがって、今回の場合、「1. より多くの個体をバルクに含める」のほうが有効である。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/L06_ans_3.png?raw=true" alt="L06_ans_3" height="350px" align="middle">

# 課題の提出について

提出期限： 2022年11月21日（月） 12:00まで

提出方法：
1. ノートブックをGoogleドライブに保存する
1. 保存後、ノートブック画面右上の「共有」ボタンから共有リンクURLを作成する
  1. アクセス設定を「リンクを知っている全員」に変更する
  1. 「リンクをコピー」をクリックする
  1. 「完了」を押して共有設定ウィンドウを閉じる
1. [Panda課題ページ](https://panda.ecs.kyoto-u.ac.jp/portal/directtool/1d1d1bd9-928d-440c-a745-9b97b2a4bc04/)のフォームに共有リンクURLを貼り付けて、課題を提出する


<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>